In [49]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.stats import poisson, uniform, expon, pareto
from scipy.optimize import minimize, fsolve
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex
import requests
import json
import time
import csv

In [3]:
start_block_height = 500000
end_block_height   = 800000
batch_size         = 10

In [5]:
500000

500000

In [ ]:
https://api.blockchair.com/bitcoin/blocks?q=id(499999..500099)&limit=100

In [6]:
url = f"https://api.blockchair.com/bitcoin/blocks?q=id(499999..500099)&limit=100"
response = requests.get(url)

In [11]:
len(response.json()['data'])

100

In [26]:
(
    response.json()['data'][-1]['id'], 
    response.json()['data'][-1]['time'], 
    response.json()['data'][-1]['fee_total'], 
    response.json()['data'][-1]['generation'],
)

(500000, '2017-12-18 18:35:25', 339351625, 1250000000)

In [22]:
blockRanges = np.arange(499999, 859726, 100)
blockRanges[0:4]

array([499999, 500099, 500199, 500299])

In [31]:
url = 'https://api.blockchair.com/bitcoin/blocks?q=id({}..{})&limit=100'.format(start, end)
response = requests.get(url)

In [51]:
newRanges = np.arange(665399, 859726, 100)
newRanges[0:4]

array([665399, 665499, 665599, 665699])

In [52]:
res = []
for i in tqdm(range(len(newRanges))):
    start, end = newRanges[i], newRanges[i+1]
    url = 'https://api.blockchair.com/bitcoin/blocks?q=id({}..{})&limit=100&key=NOPENOTTODAYXD'.format(start, end)
    response = requests.get(url)
    if response.status_code != 200:
        print(response)
        break
    data = response.json()['data']
    if len(data) != 100:
        print(data)
        break
    
    for j in range(len(data)):
        id_       = data[j]['id']
        timestamp = data[j]['time']
        fee       = data[j]['fee_total']
        blockrew  = data[j]['generation']
        res.append((id_, timestamp, fee, blockrew))
    time.sleep(0.2)

 90%|█████████ | 1754/1944 [27:33<02:59,  1.06it/s]

<Response [432]>


In [53]:
sortedOut = sorted(res, key=lambda x: x[0])

In [47]:
sortedOut[-10:]

[(665390, '2021-01-10 08:31:56', 27651681, 625000000),
 (665391, '2021-01-10 08:34:59', 32710904, 625000000),
 (665392, '2021-01-10 08:44:16', 40685876, 625000000),
 (665393, '2021-01-10 08:51:49', 40673228, 625000000),
 (665394, '2021-01-10 09:12:33', 85040585, 625000000),
 (665395, '2021-01-10 09:25:02', 55588537, 625000000),
 (665396, '2021-01-10 09:38:21', 60989980, 625000000),
 (665397, '2021-01-10 09:50:47', 52323571, 625000000),
 (665398, '2021-01-10 10:02:18', 95345152, 625000000),
 (665399, '2021-01-10 10:12:49', 61779496, 625000000)]

In [54]:
sortedOut[:10]

[(500000, '2017-12-18 18:35:25', 339351625, 1250000000),
 (500001, '2017-12-18 18:38:05', 363420702, 1250000000),
 (500002, '2017-12-18 18:41:04', 341013739, 1250000000),
 (500003, '2017-12-18 18:46:28', 363546598, 1250000000),
 (500004, '2017-12-18 18:47:21', 289758471, 1250000000),
 (500005, '2017-12-18 19:05:31', 443780733, 1250000000),
 (500006, '2017-12-18 19:22:06', 431156971, 1250000000),
 (500007, '2017-12-18 19:34:15', 434274129, 1250000000),
 (500008, '2017-12-18 19:33:56', 396853461, 1250000000),
 (500009, '2017-12-18 19:47:35', 433271130, 1250000000)]

In [50]:
file_name = 'height_time_fees_blockrew.csv'

# Writing the list of tuples to a CSV file
with open(file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write each tuple as a row in the CSV file
    for row in sortedOut:
        writer.writerow(row)

In [38]:
response.json()

{'data': None,
 'context': {'code': 402,
  'api_usage': {'requests_60s': 31,
   'requests_60s_limit': 30,
   'requests_60m': 57,
   'requests_60m_limit': 1800},
  'error': "You're sending too many requests. Please contact us at info@blockchair.com to increase the limit for you. Сurrent limits are 30 requests per minute and 1800 per hour.",
  'market_price_usd': 57786,
  'cache': {'live': True,
   'duration': 120,
   'since': '2024-09-03 18:18:54',
   'until': '2024-09-03 18:20:54',
   'time': None},
  'api': {'version': '2.0.95-ie',
   'last_major_update': '2022-11-07 02:00:00',
   'next_major_update': '2023-11-12 02:00:00',
   'documentation': 'https://blockchair.com/api/docs',
   'notice': 'Try out our new API v.3: https://3xpl.com/data'},
  'servers': 'API4',
  'time': None,
  'render_time': 0.0019161701202392578,
  'full_time': 0.0019161701202392578,
  'request_cost': 1}}

In [4]:
block_details = []
for start in range(start_block_height, end_block_height + 1, batch_size):
    end = min(start + batch_size - 1, end_block_height)

    try:
        # Blockchair API endpoint for batch block details
        url = f"https://api.blockchair.com/bitcoin/blocks?q=id(499999..500099)&limit=100"
        response = requests.get(url)

        if response.status_code == 200:
            blocks_data = response.json()['data']

            for block_data in blocks_data:
                block_time = block_data['time']
                tx_fees = block_data['fee_total']  # Fees are in Satoshis

                block_details.append({
                    "block_height": block_data['id'],
                    "block_time": block_time,
                    "transaction_fees": tx_fees
                })

                print(f"Block {block_data['id']}: Time {block_time}, Fees {tx_fees} Satoshis")

        else:
            print(f"Failed to fetch data for blocks {start} to {end}: {response.status_code}")

        # Respect API rate limits
        time.sleep(0.5)  # Adjust sleep based on API rate limits

    except Exception as e:
        print(f"An error occurred: {e}")
        continue
    break

Block 859722: Time 2024-09-03 16:50:36, Fees 3419661 Satoshis
Block 859721: Time 2024-09-03 16:48:32, Fees 5602454 Satoshis
Block 859720: Time 2024-09-03 16:41:55, Fees 3383504 Satoshis
Block 859719: Time 2024-09-03 16:39:43, Fees 4684803 Satoshis
Block 859718: Time 2024-09-03 16:37:41, Fees 4045782 Satoshis
Block 859717: Time 2024-09-03 16:35:37, Fees 7674875 Satoshis
Block 859716: Time 2024-09-03 16:19:31, Fees 5987936 Satoshis
Block 859715: Time 2024-09-03 16:14:47, Fees 9995270 Satoshis
Block 859714: Time 2024-09-03 15:57:32, Fees 3246577 Satoshis
Block 859713: Time 2024-09-03 15:56:04, Fees 5297040 Satoshis
